### Load tensorflow

In [1]:
import warnings
warnings.filterwarnings('ignore')
#!pip3 install -U tensorflow --quiet
import tensorflow as tf
from sklearn.preprocessing import Normalizer

### Collect Data

In [2]:
(trainX, trainY),(testX, testY) = tf.keras.datasets.boston_housing.load_data()

In [3]:
trainX.shape

(404, 13)

In [4]:
testX.shape

(102, 13)

In [5]:
trainX = trainX.astype('float32')
trainY = trainY.astype('float32')

In [6]:
trainX[0]

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ], dtype=float32)

In [46]:
transformer = Normalizer()
trainX = transformer.fit_transform(trainX)
testX = transformer.transform(testX)

In [47]:
trainX[0]

array([0.0024119 , 0.        , 0.01592969, 0.        , 0.00105285,
       0.01201967, 0.17945358, 0.00778265, 0.00782786, 0.60078794,
       0.04109624, 0.776719  , 0.03663436], dtype=float32)

### Build the Graph

In [48]:
# Clear the existing model in memory
tf.keras.backend.clear_session()

#Initialize Sequential Graph (model)
model = tf.keras.Sequential()

#Reshape data not needed by the way here
model.add(tf.keras.layers.Reshape((13,),input_shape=(13,)))

#Add 1st hidden layer
model.add(tf.keras.layers.BatchNormalization())                  #Normalize the data
model.add(tf.keras.layers.Dense(500, kernel_initializer='he_uniform'))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.Dropout(0.2))

#Add 2nd hidden layer
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(200, kernel_initializer='he_uniform'))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.Dropout(0.2))

#Add 3rd hidden layer
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(100, kernel_initializer='he_uniform'))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.Dropout(0.1))

#Add 4th hidden layer
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(50, kernel_initializer='he_uniform'))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.Dropout(0.1))

#Add OUTPUT layer
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(1, kernel_initializer='he_uniform'))

### Compile the Graph

In [49]:
nadam_optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam") #adam with nestrov

#Compile the model
model.compile(optimizer=nadam_optimizer, loss='mse', loss_weights=None, weighted_metrics=None,
              run_eagerly=None, steps_per_execution=None)

### Train the model

In [50]:
#%load_ext tensorboard
%load_ext tensorboard

#startting tensorboard

#%tensorboard --logdir mnist/cnn_v5 --port=8080
%tensorboard --logdir /tmp/boston --host localhost --port 8088
#%tensorboard --logdir /tmp/mnist

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 8088 (pid 756), started 0:13:46 ago. (Use '!kill 756' to kill it.)

In [51]:
# Callbacks are used to save the best model during the training, callbacks are functions which can be called during model training
model_callbacks = [tf.keras.callbacks.ModelCheckpoint('/tmp/boston/v1/boston_v1.h5', 
                                                       save_best_only=True,
                                                       monitor= 'val_loss',
                                                       mode='auto',
                                                       verbose=1),
                   tf.keras.callbacks.TensorBoard(log_dir='/tmp/boston/v1',
                                                  histogram_freq=1,
                                                  embeddings_freq=1
                                                 )
                  ]

model.fit(trainX,trainY,          
          validation_data=(testX,testY),
          epochs=100,
          callbacks=[model_callbacks],
          shuffle=True,
          use_multiprocessing=True
         )

Epoch 1/100
13/13 [==============================] - 2s 29ms/step - loss: 560.6660 - val_loss: 634.8732

Epoch 00001: val_loss improved from inf to 634.87323, saving model to /tmp/boston/v1\boston_v1.h5
Epoch 2/100
13/13 [==============================] - 0s 4ms/step - loss: 537.2667 - val_loss: 655.3227

Epoch 00002: val_loss did not improve from 634.87323
Epoch 3/100
13/13 [==============================] - 0s 5ms/step - loss: 524.5427 - val_loss: 669.4747

Epoch 00003: val_loss did not improve from 634.87323
Epoch 4/100
13/13 [==============================] - 0s 5ms/step - loss: 515.3062 - val_loss: 684.2437

Epoch 00004: val_loss did not improve from 634.87323
Epoch 5/100
13/13 [==============================] - 0s 4ms/step - loss: 502.4294 - val_loss: 711.3247

Epoch 00005: val_loss did not improve from 634.87323
Epoch 6/100
13/13 [==============================] - 0s 4ms/step - loss: 493.3361 - val_loss: 726.7842

Epoch 00006: val_loss did not improve from 634.87323
Epoch 7/100


13/13 [==============================] - 0s 4ms/step - loss: 21.2345 - val_loss: 64.0257

Epoch 00044: val_loss did not improve from 62.49686
Epoch 45/100
13/13 [==============================] - 0s 4ms/step - loss: 21.8974 - val_loss: 61.9534

Epoch 00045: val_loss improved from 62.49686 to 61.95337, saving model to /tmp/boston/v1\boston_v1.h5
Epoch 46/100
13/13 [==============================] - 0s 4ms/step - loss: 23.8397 - val_loss: 57.4012

Epoch 00046: val_loss improved from 61.95337 to 57.40123, saving model to /tmp/boston/v1\boston_v1.h5
Epoch 47/100
13/13 [==============================] - 0s 4ms/step - loss: 24.3553 - val_loss: 49.7826

Epoch 00047: val_loss improved from 57.40123 to 49.78264, saving model to /tmp/boston/v1\boston_v1.h5
Epoch 48/100
13/13 [==============================] - 0s 4ms/step - loss: 18.0241 - val_loss: 51.2896

Epoch 00048: val_loss did not improve from 49.78264
Epoch 49/100
13/13 [==============================] - 0s 6ms/step - loss: 17.5315 - val_

Epoch 94/100
13/13 [==============================] - 0s 4ms/step - loss: 12.9284 - val_loss: 19.6909

Epoch 00094: val_loss did not improve from 19.44650
Epoch 95/100
13/13 [==============================] - 0s 4ms/step - loss: 21.2033 - val_loss: 22.2823

Epoch 00095: val_loss did not improve from 19.44650
Epoch 96/100
13/13 [==============================] - 0s 4ms/step - loss: 18.7761 - val_loss: 19.3605

Epoch 00096: val_loss improved from 19.44650 to 19.36049, saving model to /tmp/boston/v1\boston_v1.h5
Epoch 97/100
13/13 [==============================] - 0s 4ms/step - loss: 17.6917 - val_loss: 22.2104

Epoch 00097: val_loss did not improve from 19.36049
Epoch 98/100
13/13 [==============================] - 0s 4ms/step - loss: 12.2517 - val_loss: 24.0901

Epoch 00098: val_loss did not improve from 19.36049
Epoch 99/100
13/13 [==============================] - 0s 4ms/step - loss: 14.6802 - val_loss: 20.4623

Epoch 00099: val_loss did not improve from 19.36049
Epoch 100/100
13/13 

In [52]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 13)                0         
_________________________________________________________________
batch_normalization (BatchNo (None, 13)                52        
_________________________________________________________________
dense (Dense)                (None, 500)               7000      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 500)               0         
_________________________________________________________________
dropout (Dropout)            (None, 500)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 500)               2000      
_________________________________________________________________
dense_1 (Dense)              (None, 200)               1

In [53]:
model.get_weights()

[array([1.0100013, 0.6767292, 0.8643492, 1.4392412, 1.4460325, 1.38519  ,
        0.7650798, 1.1616973, 1.0345279, 0.8658926, 1.0700445, 0.8702165,
        1.150055 ], dtype=float32),
 array([ 0.03861399, -0.00732674,  0.04588536, -0.0221971 ,  0.0162989 ,
        -0.04833788, -0.05402962,  0.05088482, -0.03762603, -0.01007656,
         0.07183889,  0.04054726,  0.1351386 ], dtype=float32),
 array([5.29548945e-03, 2.28651129e-02, 1.89079288e-02, 1.17134623e-04,
        9.97217256e-04, 1.15199974e-02, 1.21898584e-01, 7.16006616e-03,
        1.48405395e-02, 6.90919459e-01, 3.34735252e-02, 6.56484723e-01,
        2.22423933e-02], dtype=float32),
 array([1.54433874e-04, 2.17487221e-03, 1.14295712e-04, 2.32628145e-06,
        2.15316732e-06, 8.96807524e-06, 2.22786097e-03, 2.08625625e-05,
        1.15529416e-04, 2.20546257e-02, 3.63592771e-05, 4.43933830e-02,
        1.35992115e-04], dtype=float32),
 array([[ 0.6348724 , -0.5300675 , -0.24589057, ..., -0.27685237,
         -0.07823195, -0.2

### Save the model

In [54]:
model.save('/tmp/boston/v1/boston_v1.h5', include_optimizer=False)

In [55]:
model = tf.keras.models.load_model('/tmp/boston/v1/boston_v1.h5')

### Model Prediction





In [56]:
import numpy as np

In [57]:
testX[0].shape

(13,)

In [58]:
testY[0]

7.2

In [59]:
pred = model.predict(np.expand_dims(testX[0], axis=0))
pred

array([[10.444226]], dtype=float32)

In [61]:
pred = model.predict(np.expand_dims(testX[1], axis=0))
pred

array([[18.088037]], dtype=float32)

In [62]:
testY[1]

18.8